In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.parallel
from torch.autograd import Variable
import pandas as pd
import numpy as np
import Rbm

In [39]:
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t',header=None)
test_set = pd.read_csv('ml-100k/u1.test',delimiter='\t',header=None)

In [26]:
training_set.head(3)

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960


In [27]:
test_set.head()

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [28]:
training_set.shape

(80000, 4)

In [29]:
test_set.shape

(20000, 4)

In [40]:
training_set = np.array(training_set,dtype=int)
test_set = np.array(test_set,dtype=int)

In [31]:
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))
nb_users = int(max(max(test_set[:,0]),max(test_set[:,0])))

In [32]:
nb_movies

1682

In [34]:
nb_users

462

In [41]:
def convert(data):
    new_data = []
    for i in range(1,nb_users+1):
        mov = data[:,1][data[:,0] == i]
        rate = data[:,2][data[:,0] == i]
        rating = np.zeros(nb_movies)
        rating[mov-1] = rate
        new_data.append(list(rating))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [42]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [43]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [47]:
nv = len(training_set[0])
nh = 150
batch_size=100
epochs = 15
rbm = Rbm.RBM(nv,nh)

In [48]:
for epoch in range(1,epochs+1):
    s = 0.
    train_loss = 0
    for id in range(0,nb_users-batch_size,batch_size):
        v0 = training_set[id:id+batch_size]
        vk = training_set[id:id+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for _ in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0,vk,ph0,phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
        s += 1.
    print(f'epoch: {epoch} train loss: {train_loss/s}')

epoch: 1 train loss: 0.45462435483932495
epoch: 2 train loss: 0.26897338032722473
epoch: 3 train loss: 0.2237701267004013
epoch: 4 train loss: 0.2498195916414261
epoch: 5 train loss: 0.24405594170093536
epoch: 6 train loss: 0.23828449845314026
epoch: 7 train loss: 0.23541304469108582
epoch: 8 train loss: 0.2361031323671341
epoch: 9 train loss: 0.22208738327026367
epoch: 10 train loss: 0.23077715933322906
epoch: 11 train loss: 0.23085780441761017
epoch: 12 train loss: 0.23408544063568115
epoch: 13 train loss: 0.23008152842521667
epoch: 14 train loss: 0.23965921998023987
epoch: 15 train loss: 0.2328176200389862


In [51]:
test_loss = 0
s = 0.
for id in range(nb_users):
    v = training_set[id:id+1]
    vt = test_set[id:id+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(v[vt>=0]-vt[vt>=0]))
        s += 1.
print(f'test loss: {test_loss/s}')

test loss: 0.24813394248485565
